In [ ]:
from pathlib import Path
import os
import json

from utils import YamlConfigReader
from rag_eval import VectorDatabaseBuilder

# 约定当前工作目录为 project-root
PROJECT_ROOT = Path.cwd()
print(f"PROJECT_ROOT = {PROJECT_ROOT}")

config_path = PROJECT_ROOT / "config" / "application.yaml"
if not config_path.exists():
    raise FileNotFoundError(f"未找到配置文件：{config_path}")

config = YamlConfigReader(str(config_path))
print(f"已加载配置文件：{config.config_path}")

# 简单检查 embedding 配置和环境变量
provider = config.get("embedding.provider")
model = config.get("embedding.model")
api_key_env = config.get("embedding.api_key_env")

print(f"embedding.provider = {provider}")
print(f"embedding.model    = {model}")
print(f"embedding.api_key_env = {api_key_env}")

if not provider or not model or not api_key_env:
    raise ValueError("embedding.* 配置不完整，请检查 application.yaml")

if not os.environ.get(api_key_env):
    raise EnvironmentError(
        f"未在环境变量 {api_key_env} 中找到 API Key，"
        f"请先设置环境变量 {api_key_env}"
    )


In [ ]:
# 一键构建数据集样本 + chunks + 向量库

builder = VectorDatabaseBuilder(config)

print("开始构建向量库（包含：raw → samples → chunks → Chroma）...")

# 这里只通过 VectorDatabaseBuilder 这个入口完成所有工作
# samples_path 和 collection_name 都可以用配置中的默认值
manager = builder.invoke(
    samples_path=None,       # None → 使用 config.dataset.samples_path
    collection_name=None,    # None → 使用 config.vector_store.collection_name
    overwrite=True,          # 如已存在同名 collection，则删除后重建
)

print("向量库构建完成")
